In [ ]:
!pip install pyngrok

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import base64

## DB 접속

In [ ]:
!pip install pymysql

In [ ]:
import pymysql
import os

mysql_HOST = os.environ.get('mysql_HOST')
mysql_DBNAME = os.environ.get('mysql_DBNAME')
mysql_USER = os.environ.get('mysql_USER')
mysql_PASSWORD = os.environ.get('mysql_PASSWORD')

conn = pymysql.connect(host=mysql_HOST, user=mysql_USER, password=mysql_PASSWORD, db=mysql_DBNAME, port=mysql_PORT, charset='utf8')

## 경로 추가하기

In [ ]:
import sys
MAIN_PATH = 'your_project_path'
sys.path.append(f'{MAIN_PATH}/ElasticFace')
sys.path.append(f'{MAIN_PATH}/Pytorch_Retinaface')
sys.path.append(f'{MAIN_PATH}/deepface-emd')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/졸업프로젝트/2023-Marble-ML/Marble/ElasticFace', '/content/drive/MyDrive/졸업프로젝트/2023-Marble-ML/Marble/Pytorch_Retinaface', '/content/drive/MyDrive/졸업프로젝트/2023-Marble-ML/Marble/deepface-emd']


## 어떤 Recognition 모델 사용할 지 정하기

In [ ]:
# Recognition model 선정 "elastic" 또는 "emd"
my_model = "emd"
if my_model == "emd":
  image_size = (128, 128)
else:
  image_size = (112, 112)

# Face Detection

### detection 모델 로드하기

In [ ]:
from detect_faces import Face_Dectector, save_img, crop_alignment, handle_korean_file_name
from models.retinaface import RetinaFace


trained_model = MAIN_PATH + '/Pytorch_Retinaface/weights/Resnet50_Final.pth'
# img_path = MAIN_PATH + "/celebrity_dataset/차은우14.jpg"
network = "resnet50"
align=True

# img_array = handle_korean_file_name(img_path) # 한글 파일 이름 처리

detector = Face_Dectector(
    trained_model=trained_model,
    network=network)

cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading pretrained model from /content/drive/MyDrive/졸업프로젝트/2023-Marble-ML/Marble/Pytorch_Retinaface/weights/Resnet50_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:456
Finished loading model!


In [ ]:
def get_face(img):
  dets = detector.detect(img)
  # crop & alignment
  b = max(dets, key= lambda x: x[4])
  face = crop_alignment(img, b, align=True)[:, :, ::-1]
  # plt.imshow(face)
  # plt.show()

  # dets = detector.detect(img_array)
  # # crop & alignment
  # faces = []
  # vis_thres = 0.9
  # for b in dets:
  #   if b[4] < vis_thres:  # score
  #       continue
  #   faces.append(crop_alignment(img_array, b))
  #   print("len(faces)", len(faces))

  # # 시각화
  # for face in faces:
  #     plt.imshow(face)
  #     plt.axis('off')
  #     plt.show()

  # return faces[0]
  return face

# Face recognition - DeepFace EMD

## 의존성(Dependencies)

In [ ]:
!pip install torch>=1.0 opencv-python-headless>=3.4.4 tqdm

## Config & 모델 로드

In [ ]:
import os
import argparse
import torch
import torch.multiprocessing
from tqdm import trange
from PIL import Image

torch.multiprocessing.set_sharing_strategy('file_system')

from face_models.resnet import *
from face_models.net_cos import *
from face_models.facenet import InceptionResnetV1
from utils_emd.emd import emd_similarity
from utils_emd.metrics import get_metrics_rank
from utils_emd.extract_features import extract_embedding
from data_loader.facedata_loader import get_face_dataloader


### config
'''
python test_face.py -method apc -fm arcface -d lfw -a 0.7 -data_folder data -l 4 -crop
'''
method = 'apc'
fm = 'arcface'
alpha = 0.9
l = 4

if fm == 'arcface':
    size = (128, 128)

if fm == 'arcface':
    model_path =  f'{MAIN_PATH}/deepface-emd/pretrained/resnet18_110.pth'
    print('model : {}'.format(model_path))
    model_emd = resnet_face18(False, use_reduce_pool=False)
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))

    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove module.
        new_state_dict[name] = v
    model_emd.load_state_dict(new_state_dict)

model_emd.eval()
model_emd = nn.DataParallel(model_emd)
model_emd = model_emd.cuda()

model : /content/drive/MyDrive/졸업프로젝트/2023-Marble-ML/Marble/deepface-emd/pretrained/resnet18_110.pth


# Face recognition - Elastic

## 의존성(Dependencies)

In [ ]:
!pip install -r /content/drive/MyDrive/졸업프로젝트/2023-Marble-ML/RealTimeFaceRecognition/ElasticFace/requirement.txt

## Import packages

In [ ]:
import os
import sys
import torch
import os
import matplotlib.pyplot as plt
import numpy as np
import warnings
import time
import cv2

# 경고 메시지 무시
warnings.filterwarnings("ignore")

## Configure

In [ ]:
from config.config import config as cfg

from backbones.iresnet import iresnet100, iresnet50

gpu_id = 0
output_folder = "/content/drive/MyDrive/졸업프로젝트/2023-Marble-ML/RealTimeFaceRecognition/ElasticFace/output/R100_ElasticArcFace"
weights=os.listdir(output_folder)
print(weights)

for w in weights:
    if "backbone" in w:
        print(w)
        print(cfg.network)
        if cfg.network == "iresnet100":
            backbone = iresnet100(num_features=cfg.embedding_size).to(f"cuda:{gpu_id}")
        elif cfg.network == "iresnet50":
            backbone = iresnet50(num_classes=cfg.embedding_size).to(f"cuda:{gpu_id}")
        backbone.load_state_dict(torch.load(os.path.join(output_folder,w)))
        model_elastic = torch.nn.DataParallel(backbone, device_ids=[gpu_id])

model_elastic.eval()

['training.log', '295672backbone.pth']
295672backbone.pth
iresnet100


DataParallel(
  (module): IResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=64)
    (layer1): Sequential(
      (0): IBasicBlock(
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (prelu): PReLU(num_parameters=64)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

# 이미지 전처리

In [ ]:
# 이미지 url를 PIL로 변환

from PIL import Image
import requests
from io import BytesIO

# def imageurl_to_pil(image_url):
#   # Get the image data from the URL
#   res = requests.get(image_url)
#   img = Image.open(BytesIO(res.content))
#   return img

def imageurl_to_pil2(image_url):
  # Get the image data from the URL
  res = requests.get(image_url)
  img = Image.open(BytesIO(res.content))
  img = img.rotate(-90).transpose(Image.FLIP_LEFT_RIGHT)
  # img = img.transpose(Image.FLIP_LEFT_RIGHT)
  # plt.imshow(img)
  img = np.array(img)

  ## face detection
  # img = get_face(img)

  if my_model == "emd":
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  img = Image.fromarray(img)
  return img

In [ ]:
# 이미지 base64를 PIL로 변환

import base64
import io
from PIL import Image
import cv2

def base64_to_pil(base64_string):
    imgdata = base64.b64decode(base64_string)
    dataBytesIO = io.BytesIO(imgdata)
    image = Image.open(dataBytesIO)
    if my_model == "emd":
      image = np.array(image)
      image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
      image = Image.fromarray(image)
    return image

In [ ]:

# 이미지를 입력 받아서 feature를 반환하는 코드

from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import datetime

def my_transforms(img_path, image_size, f):
    transform = transforms.Compose([
        transforms.Resize(image_size[0]),
        transforms.CenterCrop(image_size),  # 이미지 크기를 조정하되, 원본 비율을 유지
        transforms.ToTensor(),          # 이미지를 텐서로 변환
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
    ])
    transform_emd = transforms.Compose([
        transforms.Resize(image_size[0]),
        transforms.CenterCrop(image_size),  # 이미지 크기를 조정하되, 원본 비율을 유지
        transforms.ToTensor(),          # 이미지를 텐서로 변환
        transforms.Normalize(mean=[0.5], std=[0.5])  # 정규화 (평균 0.5, 표준편차 0.5)
    ])
    try:
      if f == 'img_url':
        img = imageurl_to_pil2(img_path)


      elif f == 'base64':
        img = base64_to_pil(img_path)
      else:
        raise Exception('이미지의 타입이 불분명합니다.')
      # plt.imshow(np.array(img))
      # plt.show()

      if my_model == "emd":
        img_tensor = transform_emd(img)
      else:
        img_tensor = transform(img)
      # print(img_tensor.shape)

    except Exception as e:                             # 예외가 발생했을 때 실행됨
      print('예외가 발생했습니다: ', e)

    return img_tensor

def get_vector_from_img(img_path, f):
  # img_path = "/content/drive/MyDrive/졸업프로젝트/2023-Marble-ML/myface/myface4.jpg"  # 이미지 파일 경로
  img_tensor = my_transforms(img_path, image_size, f)
  img_tensor = img_tensor.to('cuda')

  # print(img_tensor.shape)
  img_tensor = torch.unsqueeze(img_tensor, dim=0)
  # print(img_tensor.shape)

  time0 = datetime.datetime.now()

  ### DeepFace-EMD
  if my_model == "emd":
    feature_bank_query, feature_bank_center_query, avgpool_bank_center_query = extract_embedding(img_tensor, model_emd, fm=fm, level=l)

    time_now = datetime.datetime.now()

    diff = time_now - time0
    # print(diff)

    return feature_bank_query, feature_bank_center_query, avgpool_bank_center_query

  ### ElasticFace
  else:
    net_out: torch.Tensor = model_elastic(img_tensor)
    camera_embeddings = net_out.detach().cpu().numpy()

    time_now = datetime.datetime.now()

    diff = time_now - time0
    print(diff)

    return camera_embeddings

In [ ]:
from numpy import dot
from numpy.linalg import norm

# 유사도 정의 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def get_db_vectors():
  # 데이터베이스에서 데이터 가져오기
  conn = pymysql.connect(host=mysql_HOST, user=mysql_USER, password=mysql_PASSWORD, db=mysql_DBNAME, port=mysql_PORT, charset='utf8')
  with conn.cursor() as cur:
      cur.execute("select image_id, vector from image;")
      result = cur.fetchall()
      # for data in result:
      #     print(data)
  return result

In [ ]:
def is_recognized_id(image_index, base64_image):

  # threshold 정하기
  threshold = 0.4

  # threshold 넘는 id 리스트
  result = []

  # base64 읽고 임베딩 벡터 구하기
  if my_model == "emd":
    feature_bank_gallery, feature_bank_center_gallery, avgpool_bank_center_gallery = get_vector_from_img(base64_image, f="base64")
    feat = [feature_bank_gallery, feature_bank_center_gallery, avgpool_bank_center_gallery]

    # DB에서 벡터와 ID 가져와서 누구와 유사한지 검사
    db_result_list = get_db_vectors()
    for db_row in db_result_list:
      img_id = db_row[0]
      emb_db = db_row[1]
      if emb_db:
        emb_db = eval(emb_db)
        tensor_feat = list(map(lambda x: torch.tensor(x).cuda(), emb_db))
        approx_sim, _, _, _ = emd_similarity(None, feat[1][0], None, tensor_feat[1], 0)
        sim_avg, _, _, _ = emd_similarity(feat[0][0], feat[2][0], tensor_feat[0], tensor_feat[2], 1, method=method)
        sim_torch = alpha * sim_avg + (1.0 - alpha) * approx_sim
        sim = sim_torch.tolist()[0]
        # print(f"check db_img_id: {img_id}, sim: {sim}")
        if sim > threshold:
          print(sim)
          result.append(img_id)
          return True, result
      else:
        continue

  else:
    emb_cam = get_vector_from_img(base64_image, f="base64")

    # DB에서 벡터와 ID 가져와서 누구와 유사한지 검사
    db_result_list = get_db_vectors()
    for db_row in db_result_list:
      img_id = db_row[0]
      emb_db = db_row[1]
      if emb_db:
        emb_db = eval(emb_db)
        sim = cos_sim(emb_db, emb_cam.reshape(-1,1))[0, 0]
        print(f"check db_img_id: {img_id}, sim: {sim}")
        if sim > threshold:
          result.append(img_id)
          return True, result
      else:
        continue

  return False, result

# Flask

In [ ]:
from flask import Flask, jsonify, request
from pyngrok import ngrok

ngrok.set_auth_token("your_token")

In [ ]:
from flask import Flask, request, jsonify
import base64
import numpy as np
import cv2
import matplotlib.pyplot as plt

app = Flask(__name__)

@app.route('/get_feature', methods=['POST'])
def get_feature():
  try:
    request_data = request.get_json()
    # print("request_data", request_data)
    # print(type(request_data))
    url_data = request_data.get("url")
    # print("url_data", url_data)
    # print(type(url_data))

    f = "img_url"

    if my_model == "emd":
      feature_bank_gallery, feature_bank_center_gallery, avgpool_bank_center_gallery = get_vector_from_img(url_data , f)
      feat = [feature_bank_gallery, feature_bank_center_gallery, avgpool_bank_center_gallery]
      feat_ = list(map(lambda x: x.tolist(), feat))
      emb_vector_dic = {"vector": feat_}
      # print(emb_vector_dic)

    else:
      emb_vector = get_vector_from_img(url_data , f)
      emb_vector_list = emb_vector.tolist()
      emb_vector_dic = {"vector": emb_vector_list}
      # print(emb_vector_dic)

    return jsonify(emb_vector_dic)

  except Exception as e:
        print(str(e))
        return jsonify({"error": str(e)}), 500

@app.route('/process_frame', methods=['POST'])
def process_frame():
    try:
        frames_data = request.get_json()
        # print("frames_data", frames_data)
        # print(type(frames_data))
        if frames_data is None:
            return jsonify({"error": "Invalid JSON data"}), 400

        values = frames_data.get("values")
        # print("values", values)
        # print(type(values))
        if values is None:
            return jsonify({"error": "Missing 'values' field in JSON data"}), 400


        result = []
        # 얼굴 하나씩
        for frame_data in values:
            name_value_pairs = frame_data.get("nameValuePairs")
            if name_value_pairs is None:
                print("Missing 'nameValuePairs' in frame_data")
                continue

            image_index = name_value_pairs.get("id")
            base64_image = name_value_pairs.get("image")


            if image_index is not None and base64_image is not None:
                try:
                  # Process the image and get ID
                  # 이미지를 넣고 DB에 존재하는 아이디 출력하기

                  recognized_result = is_recognized_id(image_index, base64_image)  # Implement your own ID generation function
                  # print("recognized_result", recognized_result)
                  # DB에 존재한다면
                  if recognized_result[0]: # True
                    print(f"ID: {image_index}은 DB에 ID: {recognized_result[1]}로 저장되었습니다.")
                    result.append(image_index)
                except Exception as image_exception:
                  print(f"Error processing image: {str(image_exception)}")

            else:
                print("Missing image_index or base64_image in frame_data")

        id_list = {"id_list": result}
        print(id_list)
        return jsonify(id_list)

    except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        print(error_message)
        # print("frames_data", print(frames_data))
        # print(type(frames_data))
        return jsonify({"error": error_message}), 500

app.run(port=80)